# Cellular sheaves on graphs 
## Learning sheaf laplacian through minimum total variation approach 

________________

### Generating a toy-case topology

In [1]:
import numpy as np 

In [31]:
# Let's generate a toy topology for our example

nodes = [i for i in range(7)]
edges = [
    (0,1),
    (0,3),
    (0,6),
    (1,2),
    (1,5),
    (2,4),
    (4,6),
    (5,6)
]

V = 7
E = len(edges)

d = 10                                          # Node and edges stalks dimension

F = {
    e:{
        e[0]:np.random.randn(d,d),
        e[1]:np.random.randn(d,d)
        } 
        for e in edges
    }                                           # Incidency linear maps

# Sheaf representation 

# Coboundary map

B = np.zeros((d*E, d*V))

for i in range(len(edges)):
    edge = edges[i]

    u = edge[0] 
    v = edge[1] 

    B_u = F[edge][u]
    B_v = F[edge][v]

    B[i*d:(i+1)*d, u*d:(u+1)*d] = B_u
    B[i*d:(i+1)*d, v*d:(v+1)*d] = - B_v

# Sheaf Laplacian

L_f = B.T @ B

# Generating a smooth signals dataset 

*(from Hansen J., "Learning sheaf Laplacians from smooth signals")* 

In order to retrieve a dataset of smoothsignals, first of all we sample random gaussians vectors on the nodes of the graph. Then we smooth them according to their expansion in terms of the eigenvectors of the sheaf Laplacian $L_0$.

So let's firstly define a dataset of random gaussian vectors. 

In [32]:
N = 100
X = np.random.randn(V*d,N)

Now we'll use the Fourier-domain embedded in the Laplacian spectrum. 

We'll consider a Tikhonov inspired procedure where we firstly project our dataset over the space spanned by the eigenvectors of the sheaf laplacian: namely $U$ the matrix collecting this eigenvectors we have 
\begin{equation}
    \hat{x} = U^T x
\end{equation}

So that defining $h(\lambda) = \frac{1}{1 + 10\lambda}$ and $H = \mathrm{diag}\{h(\lambda)\}_{\lambda}$, we now have

\begin{equation}
    \hat{y} = H(\Lambda) \hat{x}
\end{equation}

and finally our dataset is just reprojected back into the vertex domain:

\begin{equation}
    y = U H(\Lambda) \hat{x} = U H(\Lambda) U^T x
\end{equation}

In [33]:
Lambda, U = np.linalg.eig(L_f)
H = 1/(1 + 10*Lambda)

In [34]:
Y = U @ np.diag(H) @ U.T @ X

Y += np.random.normal(0, 10e-2, size=Y.shape)

In [35]:
np.trace(X.T @ L_f @ X)

147278.39956785546

In [36]:
np.trace(Y.T @ L_f @ Y)

1572.0416740350815

_________________________

### A first test

Let's try our centralized procedure over our toy case topology. 

In [37]:
from controller import learning
from itertools import combinations

edge_blocks = learning(7, d, Y,  0.005, 30, 50)

100%|██████████| 50/50 [00:00<00:00, 270.10it/s]


Let's now retrieve the energy expressed by each edge:

In [38]:
all_edges = list(combinations(range(V), 2))

energies = {
    e : 0
    for e in all_edges
    }

for e in (all_edges):
    BB = edge_blocks[e]
    u = e[0]
    v = e[1]
    
    '''
    X_ = np.zeros_like(X)
    X_[u*d:(u+1)*d,:] = X[u*d:(u+1)*d,:]
    X_[v*d:(v+1)*d,:] = X[v*d:(v+1)*d,:]
    energies[e] = np.linalg.norm(BB @ X_)
    '''

    energies[e] = np.linalg.norm(BB @ X)

We'll consider the first $E_0$ edges sorted accordingly to their energy:

In [39]:
retrieved = sorted(energies.items(), key=lambda x:x[1])[:E]

Let's reconstruct the sheaf laplacian:

In [40]:
L_ = np.zeros((d*V, d*V))

for i in range(E):
    edge = retrieved[i][0]

    BB_ = edge_blocks[edge]

    L_ += BB_

In [41]:
# The metric chosen by Hansen for the evaluation was the average entry-wise euclidean distance

np.sqrt(np.sum((L_f - L_)**2)) / L_f.size

2.8005282586706742e+112

Let's see the precision of our procedure:

In [42]:
len(set(list(map(lambda x: x[0], retrieved))).intersection(set(edges))) / E

0.375

In [43]:
edges

[(0, 1), (0, 3), (0, 6), (1, 2), (1, 5), (2, 4), (4, 6), (5, 6)]

In [23]:
retrieved

[((1, 5), 1.7110634493927492e+24),
 ((0, 5), 1.7538085612326358e+24),
 ((1, 6), 1.8378374544102506e+24),
 ((2, 3), 1.841757661654543e+24),
 ((3, 5), 1.9383096878678974e+24),
 ((1, 2), 1.973720967333e+24),
 ((2, 5), 2.065004998102306e+24),
 ((0, 2), 2.113891994867891e+24)]